In [21]:
import tensorflow as tf
from tensorflow.keras.layers import Input, ConvLSTM2D, GlobalAveragePooling2D, Conv3D, Flatten, Dense, Dropout, LayerNormalization, GlobalAveragePooling3D, Concatenate, TimeDistributed, MaxPooling3D, AveragePooling3D, GlobalMaxPooling3D, LSTM, Lambda
from tensorflow.keras.models import Model
import numpy as np
import pickle as pkl

In [22]:
devices = tf.config.list_physical_devices()
print(devices)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


In [3]:
tf.config.set_visible_devices([], 'GPU')

In [ ]:
# importing data
def returning_pkl_file_data(path : str):
    with open(path, 'rb') as f:
        temp = pkl.load(f)
    return temp

# mci_func = returning_pkl_file_data(r"feature_extraction/MCI_func_52_79_95_79_197.pkl")
mci_struct = returning_pkl_file_data(r'feature_extraction/MCI_struct_cat_52_169_205_169.pkl')
# cn_func = returning_pkl_file_data(r'feature_extraction/CN_func_42_79_95_79_197.pkl')
cn_struct = returning_pkl_file_data(r'feature_extraction/CN_struct_cat_42_169_205_169.pkl')

In [ ]:
# func_data = np.concat((mci_func, cn_func), axis=0, dtype=np.float32)
# func_data.shape

(94, 79, 95, 79, 197)

In [6]:
struct_data = np.concat((mci_struct, cn_struct), axis=0, dtype=np.float32)
struct_data.shape

(94, 169, 205, 169)

In [ ]:
all_labels = np.concat((np.zeros((len(mci_struct),)), np.ones((len(cn_struct),))))
all_labels.shape

(94,)

In [ ]:
# func_data = np.expand_dims(func_data, axis=len(func_data.shape))
struct_data = np.expand_dims(struct_data, axis=len(struct_data.shape))

struct_data.shape, #func_data.shape

((94, 79, 95, 79, 197), (94, 169, 205, 169, 1))

In [ ]:
from sklearn.model_selection import train_test_split

struct_train, struct_test, y_train, y_test = train_test_split(
    struct_data, all_labels, test_size=0.2, random_state=42
)

struct_train.shape, struct_test.shape, y_train.shape, y_test.shape

((75, 79, 95, 79, 197),
 (19, 79, 95, 79, 197),
 (75, 169, 205, 169, 1),
 (19, 169, 205, 169, 1),
 (75,),
 (19,))

In [28]:
def build_smri_model(smri_shape=(struct_data.shape[1:])):    
    # sMRI model (3D CNN)
    smri_input = layers.Input(shape=smri_shape, name="smri_input", dtype=tf.float32)
    y = Conv3D(32, 3, activation="relu", padding="same")(smri_input)
    y = MaxPooling3D(2)(y)
    y = Conv3D(64, 3, activation="relu", padding="same")(y)
    y = MaxPooling3D(2)(y)
    y = Conv3D(128, 3, activation="relu", padding="same")(y)
    y = MaxPooling3D(2)(y)
    y = GlobalAveragePooling3D()(y)
    y = Dense(256, activation="relu")(y)
    y = Dropout(0.5)(y)
    y = Dense(128, activation="relu")(y)
    y = Dropout(0.5)(y)
    output = Dense(1, activation="sigmoid")(y)
    return Model(inputs=smri_input, outputs=output, name="sMRI_Model")
    
tf.keras.mixed_precision.set_global_policy('float32')
# Build and compile model
model = build_smri_model()
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

Model: "sMRI_Model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 smri_input (InputLayer)     [(None, 10, 10, 10, 1)]   0         
                                                                 
 conv3d_9 (Conv3D)           (None, 10, 10, 10, 32)    896       
                                                                 
 max_pooling3d_6 (MaxPoolin  (None, 5, 5, 5, 32)       0         
 g3D)                                                            
                                                                 
 conv3d_10 (Conv3D)          (None, 5, 5, 5, 64)       55360     
                                                                 
 max_pooling3d_7 (MaxPoolin  (None, 2, 2, 2, 64)       0         
 g3D)                                                            
                                                                 
 conv3d_11 (Conv3D)          (None, 2, 2, 2, 128)      2

In [23]:
# func_data = np.random.rand(100, 4, 4, 4, 5).astype(np.float32)
struct_data = np.random.rand(100, 10, 10, 10, 1).astype(np.float32)
all_labels = np.random.randint(0, 2, size=(100,))
struct_data.shape, all_labels.shape#, func_data.shape 

((100, 10, 10, 10, 1), (100,))

In [24]:
from sklearn.model_selection import train_test_split

In [25]:
struct_train, struct_test, y_train, y_test = train_test_split(struct_data, all_labels, test_size=0.2, random_state=42)
struct_train.shape, struct_test.shape, y_train.shape, y_test.shape

((80, 10, 10, 10, 1), (20, 10, 10, 10, 1), (80,), (20,))

In [30]:
history = model.fit(
    {"smri_input": struct_train},  # Dictionary format for inputs
    y_train,  # Output labels
    batch_size=8,
    epochs=10,
    validation_data=(
        {"smri_input": struct_test},
        y_test
    ),
    verbose=1
)

Epoch 1/10
10/10 [==============================] - 1s 37ms/step - loss: 0.6931 - accuracy: 0.5375 - val_loss: 0.6943 - val_accuracy: 0.5000
Epoch 2/10
10/10 [==============================] - 0s 16ms/step - loss: 0.6893 - accuracy: 0.5750 - val_loss: 0.6950 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 0s 15ms/step - loss: 0.6849 - accuracy: 0.5500 - val_loss: 0.6971 - val_accuracy: 0.5000
Epoch 4/10
10/10 [==============================] - 0s 15ms/step - loss: 0.6828 - accuracy: 0.5375 - val_loss: 0.6961 - val_accuracy: 0.5000
Epoch 5/10
10/10 [==============================] - 0s 16ms/step - loss: 0.6759 - accuracy: 0.5625 - val_loss: 0.7034 - val_accuracy: 0.5000
Epoch 6/10
10/10 [==============================] - 0s 15ms/step - loss: 0.6529 - accuracy: 0.5500 - val_loss: 0.7001 - val_accuracy: 0.5000
Epoch 7/10
10/10 [==============================] - 0s 16ms/step - loss: 0.6024 - accuracy: 0.5750 - val_loss: 0.7899 - val_accuracy: 0.5000
Epoch 8/10
10